In [1]:
# DELIVERABLE FOR THE PROJECT:
# "KINEMATICS OF MOBILITY"
# CIMMYT 
# David Pastor-Escuredo (Life D Lab)
# Licencia MIT

In [2]:
#Copyright <2019> <David Pastor Escuredo>

#Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

#The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

#THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [23]:
import pandas as pd
import numpy as np
import json
from nltk.tokenize import word_tokenize
import re
import networkx as nx
import matplotlib.pyplot as plt
import datetime
import collections
from sklearn.cluster import KMeans
from PIL import Image

def getday(d):
    if d<10:
        ds='0'+str(d)
    else:
        ds=str(d)
    return ds

In [36]:
ant_file='antennas/antennas.csv'
sheet=pd.read_csv(ant_file,delimiter=';')

ant_file_col='antennas/antennas_colombia.csv'
sheet2=pd.read_csv(ant_file_col,delimiter=',')

ant_file_b='antennas/antennas_bogota.csv'
sheet3=pd.read_csv(ant_file_b,delimiter=',')

print(len(sheet))
print(len(sheet2))
print(len(sheet3))

print(sheet.head())
print(sheet2.head())
print(sheet3.head())

29869
29865
6757
   LATITUD  LONGITUD Celda_HEX LAC_HEX
0   4.6092  -74.0718      2711    083F
1   4.6092  -74.0718      2712    083F
2   4.6092  -74.0718      2713    083F
3   4.6092  -74.0718      2714    083F
4   4.6092  -74.0718      2715    083F
   index  LATITUD  LONGITUD Celda_HEX LAC_HEX antenna_id
0      0   4.6092  -74.0718      2711    083F   083F2711
1      1   4.6092  -74.0718      2712    083F   083F2712
2      2   4.6092  -74.0718      2713    083F   083F2713
3      3   4.6092  -74.0718      2714    083F   083F2714
4      4   4.6092  -74.0718      2715    083F   083F2715
   Unnamed: 0  LATITUD  LONGITUD Celda_HEX LAC_HEX antenna_id    municipio
0           0   4.6092  -74.0718      2711    083F   083F2711  BOGOTÁ D.C.
1           1   4.6092  -74.0718      2712    083F   083F2712  BOGOTÁ D.C.
2           2   4.6092  -74.0718      2713    083F   083F2713  BOGOTÁ D.C.
3           3   4.6092  -74.0718      2714    083F   083F2714  BOGOTÁ D.C.
4           4   4.6092  -74.0718

In [30]:
ant_file='/Volumes/OS Disk/Tef_Colombia/datos/CONSULTA_SECTORES.xls'
ant_file_mun='antennas/CONSULTA_SECTORES_2.xls'
ant_file='antennas/antennas.csv'
xls=pd.ExcelFile(ant_file_mun)#,decimal=',')
sheetMun=xls.parse(0)
print(len(sheetMun))
mun=sheetMun['COD_DANE_MUNICIPIO']

sheet=pd.read_csv(ant_file,delimiter=';')
print(len(sheet))
print(sheet.head())
LAC=sheet['LAC_HEX']
Cell=sheet['Celda_HEX']
lon=sheet['LONGITUD']
lat=sheet['LATITUD']
sheet['antenna_id']=LAC+Cell
sheet['municipio']=mun
#print(sheet['antenna_id'])
aid=sheet['antenna_id']
print(len(sheet))

minlat=np.float64(np.min(lat))
maxlat=np.float64(np.max(lat))
minlon=np.float64(np.min(lon))
maxlon=np.float64(np.max(lon))
print(minlat,maxlat,minlon,maxlon)


29869
29869
   LATITUD  LONGITUD Celda_HEX LAC_HEX
0   4.6092  -74.0718      2711    083F
1   4.6092  -74.0718      2712    083F
2   4.6092  -74.0718      2713    083F
3   4.6092  -74.0718      2714    083F
4   4.6092  -74.0718      2715    083F
29869
-76.5484 13.3764 -81.7294 74.0386


In [25]:
mun_data=sheet[sheet['municipio']=='MEDELLIN']
print(len(mun_data))
sheet=mun_data
LAC=sheet['LAC_HEX']
Cell=sheet['Celda_HEX']
lon=sheet['LONGITUD']
lat=sheet['LATITUD']
aid=sheet['antenna_id']

sheet.to_csv('antennas/antennas_medellin.csv')

1352


In [28]:
mun_data=sheet[sheet['municipio']=='BOGOTÁ D.C.']
print(len(mun_data))
sheet=mun_data
LAC=sheet['LAC_HEX']
Cell=sheet['Celda_HEX']
lon=sheet['LONGITUD']
lat=sheet['LATITUD']
aid=sheet['antenna_id']

sheet.to_csv('antennas/antennas_bogota.csv')

6757


In [29]:
#lat=pd.to_numeric(lat)
#lon=pd.to_numeric(lon)
minlat=np.float64(np.min(lat))
maxlat=np.float64(np.max(lat))
minlon=np.float64(np.min(lon))
maxlon=np.float64(np.max(lon))
print(minlat,maxlat,minlon,maxlon)

minlon=-83
minlat=0

sheet2=sheet[sheet['LONGITUD']<0]
sheet3=sheet2[sheet2['LATITUD']>0]

print(len(sheet))
print(len(sheet2))
print(len(sheet3))

LAC=sheet3['LAC_HEX']
Cell=sheet3['Celda_HEX']
lon=sheet3['LONGITUD']
lat=sheet3['LATITUD']
aid=sheet3['antenna_id']

minlat=np.float64(np.min(lat))
maxlat=np.float64(np.max(lat))
minlon=np.float64(np.min(lon))
maxlon=np.float64(np.max(lon))
print(minlat,maxlat,minlon,maxlon)

4.47804 4.8147199999999994 -74.207 74.0386
6757
6756
6756
4.47804 4.8147199999999994 -74.207 -74.0227
